In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#필요한 라이브러리 import
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.model_selection import train_test_split
import torch.optim as optim

In [3]:
%cd /content/drive/MyDrive/Colab Notebooks
!mkdir USG_AI

/content/drive/MyDrive/Colab Notebooks
mkdir: cannot create directory ‘USG_AI’: File exists


In [4]:
#working directory 옮기기
%cd /content/drive/MyDrive/Colab Notebooks/USG_AI
!ls
# !unzip -q -o "{MAIN_DIR}/task1.zip" #현재 디렉토리에 압축풀기

/content/drive/MyDrive/Colab Notebooks/USG_AI
task1  task1.zip


In [5]:
MAIN_DIR = os.getcwd() #USG_AI까지의 경로
TASK1_DATA_DIR = MAIN_DIR + "/task1" #USG_AI/task1의 경로
TRAIN_DIR = TASK1_DATA_DIR + "/train" #task1/train
TEST_DIR = TASK1_DATA_DIR + "/test" #task1/test

In [6]:
len(os.listdir(TRAIN_DIR)), len(os.listdir(TEST_DIR)) #압축해제된 파일 개수가 맞는지 확인

(3500, 1501)

In [7]:
#개요에 명시된 test 파일 개수와 압축해제한 파일의 개수가 달라 확인

sample = pd.read_csv( TASK1_DATA_DIR+"/answer_sample.csv" )
file_name = sample["file_name"]
dir_list = os.listdir(TEST_DIR)
set(dir_list) - set(file_name) #.DS_Store 파일

{'.DS_Store'}

In [8]:
train_labels = pd.read_csv(TASK1_DATA_DIR + "/train_labels.csv")
train_file = pd.DataFrame({"file_name" : train_labels["file_name"],
                           "file_path" : [TRAIN_DIR + "/" + i for i in os.listdir(TRAIN_DIR)],
                           "labels" : train_labels["bad"].astype("float")})
train_file # train data의 이름, 경로, 라벨을 묶어 하나의 dataFrame 만들기

,file_name,file_path,labels
0,train0000.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0
1,train0001.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0
2,train0002.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0
3,train0003.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0
4,train0004.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0
...,...,...,...
3495,train3495.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0
3496,train3496.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0
3497,train3497.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0
3498,train3498.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0


In [9]:
SEED = 42
np.random.seed(SEED)
status = ["train" if i <= 0.8 else "val" for i in np.random.uniform(low=0.0, high=1.0, size=(train_file.shape[0]))]
train_file["status"] = status
train_file["status"].value_counts(normalize = True)

,proportion
status,
train,0.800571
val,0.199429


In [10]:
val_file = train_file[train_file["status"] == "val"]
val_file

,file_name,file_path,labels,status
1,train0001.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0,val
7,train0007.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0,val
11,train0011.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,val
12,train0012.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0,val
33,train0033.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0,val
...,...,...,...,...
3483,train3483.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,val
3492,train3492.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,val
3493,train3493.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,val
3496,train3496.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0,val


In [11]:
train_file = train_file[train_file["status"] == "train"]
train_file

,file_name,file_path,labels,status
0,train0000.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0,train
2,train0002.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0,train
3,train0003.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,train
4,train0004.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,train
5,train0005.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,train
...,...,...,...,...
3491,train3491.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0,train
3494,train3494.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,train
3495,train3495.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,train
3498,train3498.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,train


In [12]:
test_file = pd.DataFrame({"file_name" : [i for i in os.listdir(TEST_DIR) if i != ".DS_Store" ],
                           "file_path" : [TEST_DIR + "/" + i for i in os.listdir(TEST_DIR) if i != ".DS_Store" ]})

test_file #test 파일이름, 경로 dataFrame


,file_name,file_path
0,test0000.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...
1,test0001.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...
2,test0002.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...
3,test0003.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...
4,test0004.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...
...,...,...
1495,test1495.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...
1496,test1496.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...
1497,test1497.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...
1498,test1498.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...


In [13]:
#데이터 셋 정의
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform = None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        data_row = self.dataframe.iloc[idx]
        image_path = data_row["file_path"]
        image = Image.open(image_path).convert("RGB")
        label = data_row["labels"]

        if self.transform:
            image = self.transform(image)

        return image, label

In [14]:
#train_transform
#train_transform은 데이터 증강(data augmentation) 해야함
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),# 50% 확률로 좌우 반전
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=30),   # -15도에서 +15도 범위로 랜덤 회전
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # 밝기, 대비, 채도, 색조 조정
    transforms.ToTensor(),  # 텐서 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


#test_transform 만들어야 함
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),                   # 텐서 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


#Resize(224,224)
#ToTensor()만 하면 됨

# 데이터셋에 전처리 적용
BATCH_SIZE = 32
NUM_WORKERS = 1

train_dataset = CustomDataset(dataframe=train_file, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, num_workers=NUM_WORKERS)

val_dataset = CustomDataset(dataframe = val_file, transform = test_transform)
val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE, num_workers=NUM_WORKERS)

test_dataset = CustomDataset(dataframe = test_file, transform = test_transform)
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE, num_workers=NUM_WORKERS)

In [15]:
def show_rgb_images(loader, num_images=16):
    # DataLoader에서 한 배치 가져오기
    images, labels = next(iter(loader))

    # 4x4 그리드에 이미지 시각화
    grid_size = int(num_images ** 0.5)
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(10, 10))

    for i, ax in enumerate(axes.flat):
        if i >= num_images:
            break
        image = images[i].permute(1, 2, 0).numpy()  # [C, H, W] -> [H, W, C]

        ax.imshow(image)
        ax.set_title(f"Label: {labels[i]}")
        ax.axis('off')  # 축 제거

    plt.tight_layout()
    plt.show()

# show_rgb_images(train_loader)


In [22]:
model = models.resnet101(pretrained=True)

In [23]:
LR = 0.001

criterion = nn.BCELoss()  # 이진교차
optimizer = optim.Adam(model.parameters(), lr=LR)

In [24]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images, labels.view(-1, 1)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()  # 역전파
            optimizer.step()  # 가중치
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


In [25]:
# train_model(model, train_loader, criterion, optimizer, num_epochs=10)

In [26]:
# results_df = predict(model, test_loader)

In [27]:
def save_results(results_df):
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"task1_{current_time}.csv"
    results_df.to_csv(file_name, index=False)
    print(f"Results saved to {file_name}")


# save_results(results_df)